# Violation of Article 10 Feature Extraction

In [1]:
# TODO: PRIORITY

# Could scrape again but with the non/yes violation filter on HUDOC
# Text Preprocessing on these
# Try hugging face for feature extraction and BOW
# Dimensionality reduction
# Move what WORKS! to python classes
# Start with intro and states of the arts 

# Combine preprocess and clean functions
# QUESTION: Stemming or Lemmatization?
# QUESTION: in text preprocessing punctuation removal is not always included, what do you recommend? Maybe, like in one of the papers, it can be an experiment (which things to include in text preprocessing)

In [1]:
import pandas as pd
from transformers import pipeline
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import os

In [2]:
data = pd.read_csv('violation_or_not.csv')
data.head()

,Case,Violation
0,001-223656,1
1,001-211828,1
2,001-160270,1
3,001-210075,0
4,001-203169,1


In [3]:
data['Violation'].value_counts()

1    275
0     58
Name: Violation, dtype: int64

In [4]:
violation = data.drop(data[data['Violation'] == 0].index)
violation.head()

,Case,Violation
0,001-223656,1
1,001-211828,1
2,001-160270,1
4,001-203169,1
5,001-209444,1


In [5]:
violation = violation.reset_index()

In [6]:
file = violation['Case'].iloc[0]

In [7]:
filenames = []
directories = []
for filename in os.listdir('all_ten'):
    filenames.append(filename)
    file_path = os.path.join('all_ten', filename)
    directories.append(file_path)

In [8]:
def clean(file):
    with open('all_ten/' + file + '.txt') as f:
        lines = f.readlines()
    filter_object = list(filter(lambda a: 'FIRST SECTION' in a, lines))
    if len(filter_object) == 0:
        filter_object = list(filter(lambda a: 'SECOND SECTION' in a, lines))
    if len(filter_object) == 0:
        filter_object = list(filter(lambda a: 'THIRD SECTION' in a, lines))
    if len(filter_object) == 0:
        filter_object = list(filter(lambda a: 'FOURTH SECTION' in a, lines))
    if len(filter_object) == 0:
        filter_object = list(filter(lambda a: 'FIFTH SECTION' in a, lines))
    cleaned = filter_object[0].replace('\xa0', ' ').replace('•', '').replace('§','').replace('”', '')
    return cleaned

In [9]:
# Summarization
trial_file = clean(file)
trial_file

' FIRST SECTIONCASE OF ROGALSKI v. POLAND(Application no. 5420/16)     JUDGMENTArt 10  Freedom of expression  Unjustified disciplinary sanctioning of lawyer for unethical conduct for notifying a commission of a criminal offence by a prosecutor without a proper basis in fact, moderation, proportionality and caution  Disciplinary courts’ failure to give relevant and sufficient reasons for decision  Margin of appreciation overstepped STRASBOURG23 March 2023 This judgment will become final in the circumstances set out in Article 44  2 of the Convention. It may be subject to editorial revision.In the case of Rogalski v. Poland,The European Court of Human Rights (First Section), sitting as a Chamber composed of: Marko Bošnjak, President, Péter Paczolay, Krzysztof Wojtyczek, Alena Poláčková, Erik Wennerström, Ioannis Ktistakis, Davor Derenčinović, judges,and Renata Degener, Section Registrar,Having regard to:the application (no. 5420/16) against the Republic of Poland lodged with the Court un

In [10]:
import re
# removes all non-alphanumeric characters and spaces
trial_file = re.sub(r'[^\w\s]', '', trial_file) 
# removes all digits
trial_file = re.sub(r'\d+', '', trial_file)
# makes sure there is a space between uppercase and lowercase text
trial_file = re.sub(r'(?<=[a-z0-9])(?=[A-Z])', ' ', trial_file)

In [11]:
trial_file

' FIRST SECTIONCASE OF ROGALSKI v POLANDApplication no      JUDGMENTArt   Freedom of expression  Unjustified disciplinary sanctioning of lawyer for unethical conduct for notifying a commission of a criminal offence by a prosecutor without a proper basis in fact moderation proportionality and caution  Disciplinary courts failure to give relevant and sufficient reasons for decision  Margin of appreciation overstepped STRASBOURG March  This judgment will become final in the circumstances set out in Article    of the Convention It may be subject to editorial revision In the case of Rogalski v Poland The European Court of Human Rights First Section sitting as a Chamber composed of Marko Bošnjak President Péter Paczolay Krzysztof Wojtyczek Alena Poláčková Erik Wennerström Ioannis Ktistakis Davor Derenčinović judgesand Renata Degener Section Registrar Having regard tothe application no  against the Republic of Poland lodged with the Court under Article  of the Convention for the Protection of

In [12]:
cut = trial_file.split()
start_index = cut.index('FACTS') + 1
end_index = cut.index('LAWALLEGED')
the_facts = ' '.join(cut[start_index:end_index])
the_facts

'The applicant was born in and lives in Warsaw He was represented by Ms M Gniady a lawyer practising in Warsaw The Polish Government the Government were represented by their Agent Mr J Sobczak of the Ministry of Foreign Affairs The facts of the case may be summarised as follows On January the applicant who is a lawyer lodged on behalf of his client company a formal notification of a crime that contained the following statement the data exists to sufficiently justify a suspicion that prosecutor M P full name of the prosecutor and his service had committed an offence of bribe taking by means of accepting a bribe from unknown persons who were probably employees of a company mentioned by name that is to say an offence under Article of the Criminal Code In the document the applicant explained that the offence in question had been committed in connection with an investigation into allegations of forging contracts to the detriment of his client which was ongoing at that time In particular the

In [13]:
# List of legal stopwords found using chatgpt
legal_sw = ['adjourned', 
'affidavit', 
'allegation', 
'appeal', 
'appellant', 
'application',
'applicant',
'arbitration', 
'case', 
'cause', 
'claim', 
'clerk', 
'complaint', 
'consent', 
'contempt',
'contravention',
'conviction',
'costs', 
'court', 
'cross-examination',
'defence',
'defendant', 
'deposition', 
'discovery', 
'dispute', 
'evidence', 
'examination', 
'fact', 
'hearing', 
'judge', 
'judgment', 
'jurisdiction', 
'justice', 
'law', 
'lawsuit', 
'legal', 
'litigant', 
'litigation', 
'moot', 
'motion', 
'objection', 
'order', 
'parties', 
'pleading', 
'proceedings', 
'ruling', 
'sentence', 
'settlement', 
'solicitor', 
'statute']

In [14]:
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    # Remove stop words
    all_stopwords = set(stopwords.words('english') + legal_sw)
    filtered_tokens = [token for token in tokens if token not in all_stopwords]
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    # Join the tokens back into a string
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text

In [15]:
processed_facts = preprocess_text(the_facts)

In [16]:
processed_facts

'born life warsaw represented m gniady lawyer practising warsaw polish government government represented agent mr j sobczak ministry foreign affair fact may summarised follows january lawyer lodged behalf client company formal notification crime contained following statement data exists sufficiently justify suspicion prosecutor p full name prosecutor service committed offence bribe taking mean accepting bribe unknown person probably employee company mentioned name say offence article criminal code document explained offence question committed connection investigation allegation forging contract detriment client ongoing time particular alleged request filed course abovementioned investigation examined promptly shortcoming occurred overall investigation question prematurely unjustifiably discontinued ground insufficient justify suspicion crime committed expert report obtained prosecutor view explanation irrational decision discontinue investigation mp prosecutor charge investigation ques

In [19]:
from transformers import AutoTokenizer, TFAutoModelForTokenClassification
from transformers import pipeline

In [21]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/distilbert-base-multilingual-cased-ner-hrl")
model = TFAutoModelForTokenClassification.from_pretrained("Davlan/distilbert-base-multilingual-cased-ner-hrl")

All model checkpoint layers were used when initializing TFDistilBertForTokenClassification.

All the layers of TFDistilBertForTokenClassification were initialized from the model checkpoint at Davlan/distilbert-base-multilingual-cased-ner-hrl.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForTokenClassification for predictions without further training.


In [23]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "The applicant was born in and lives in Warsaw"
ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-LOC', 'score': 0.99931586, 'index': 11, 'word': 'Warsaw', 'start': 39, 'end': 45}]


## TF-IDF - Term Frequency

TF - Term Frequency : tf(t,d) = count of t in d / number of words in d. 
- t: word
- d: document
- N: number of documents

In [105]:
# For every word in the text
# if word has not been seen, but in list

unique_words = []
count_list = []
frequency_list = []
# Document frequency?
freq_df = pd.DataFrame(columns = ['Words', 'Count', 'T_Freq'])

list_words = processed_facts.split()
tot_words = len(list_words) # 1075

# CHECK UNIQUE WORDS IN THE DOC
for word in list_words:
    if word not in unique_words:
        unique_words.append(word)

# NUMBER OF OCCURENCES OF SPECIFIC WORD IN DOC
for unique in unique_words:
    count = 0
    for word in list_words:
        if unique == word:
            count = count+1
    count_list.append(count)

freq_df['Words'] = unique_words # 541
freq_df['Count'] = count_list

# CALCULATE FREQUENCY
freq_df['T_Freq'] = freq_df['Count']/tot_words
freq_df.head()

,Words,Count,T_Freq
0,born,1,0.000930
1,life,2,0.001860
2,warsaw,7,0.006512
3,represented,2,0.001860
4,m,1,0.000930


In [106]:
tot_words

1075

In [107]:
len(unique_words)

541

In [108]:
freq_df['Count'].sum()

1075

### Now I could do:
- Compare for each document the df with all other documents
- or have some more text preprocessing. Like remove applicant or cities etc. and do one huge list of unique words from all documents and calculate frequencies

In [109]:
# Finding most frequent words
freq_df.sort_values('T_Freq', ascending=False)

,Words,Count,T_Freq
55,criminal,17,0.015814
135,disciplinary,16,0.014884
132,professional,15,0.013953
6,lawyer,15,0.013953
37,prosecutor,13,0.012093
...,...,...,...
244,remember,1,0.000930
243,never,1,0.000930
242,would,1,0.000930
241,given,1,0.000930


# No Violation of Article 10 Feature Extraction

In [8]:
no_violation = data.drop(data[data['Violation'] == 1].index)
no_violation.head()

,Case,Violation
3,001-210075,0
9,001-186118,0
10,001-126635,0
11,001-219482,0
13,001-158467,0


In [11]:
no_violation = no_violation.reset_index()